<a href="https://colab.research.google.com/github/nerfstudio-project/nerfstudio/blob/alex%2Fcolab-logo-fix/colab/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
    <picture>
    <source media="(prefers-color-scheme: dark)" srcset="https://docs.nerf.studio/_images/logo-dark.png">
    <source media="(prefers-color-scheme: light)" srcset="https://docs.nerf.studio/_images/logo.png">
    <img alt="nerfstudio" src="https://docs.nerf.studio/_images/logo.png" width="400">
    </picture>
</p>


# Nerfstudio: A collaboration friendly studio for NeRFs


![GitHub stars](https://img.shields.io/github/stars/nerfstudio-project/nerfstudio?color=gold&style=social)

This colab shows how to train and view NeRFs from Nerfstudio both on pre-made datasets or from your own videos/images.

\\

Credit to [NeX](https://nex-mpi.github.io/) for Google Colab format.

## Frequently Asked Questions

*  **Downloading custom data is stalling (no output):**
    * This is a bug in Colab. The data is processing, but may take a while to complete. You will know processing completed if `data/nerfstudio/custom_data/transforms.json` exists. Terminating the cell early will result in not being able to train.
*  **Processing custom data is taking a long time:**
    * The time it takes to process data depends on the number of images and its resolution. If processing is taking too long, try lowering the resolution of your custom data.
*  **Error: Data processing did not complete:**
    * This means that the data processing script did not fully complete. This could be because there were not enough images, or that the images were of low quality. We recommend images with little to no motion blur and lots of visual overlap of the scene to increase the chances of successful processing.
*   **Training is not showing progress**:
    * The lack of output is a bug in Colab. You can see the training progress from the viewer.
* **Viewer Quality is bad / Low resolution**:
    * This may be because more GPU is being used on training that rendering the viewer. Try pausing training or decreasing training utilization.
* **WARNING: Running pip as the 'root' user...:**:
    * This and other pip warnings or errors can be safely ignored.
* **Other problems?**
    * Feel free to create an issue on our [GitHub repo](https://github.com/nerfstudio-project/nerfstudio).


In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
# Installing PyTorch and others (takes 3 minutes)
%cd /content/drive/MyDrive/workspace/nerfstudio/1/
!pip install --upgrade pip
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 --extra-index-url https://download.pytorch.org/whl/cu118

# Installing TinyCuda (takes 10 seconds)
%cd /content/drive/MyDrive/workspace/nerfstudio/1/
!gdown "https://drive.google.com/u/1/uc?id=1-7x7qQfB7bIw2zV4Lr6-yhvMpjXC84Q5&confirm=t"
!pip install tinycudann-1.7-cp310-cp310-linux_x86_64.whl

# Downloading COLMAP binary executable and installing its dependencies (takes 10 seconds)
%cd /usr/local/bin
!gdown "https://drive.google.com/file/d/1rgIHYDNKPytiYUEFeKbPuRQf_zVs-qOE/view?usp=sharing" --fuzzy
!chmod +x /usr/local/bin/colmap

# Downloading and installing libglvnd to fix OpenGl path (takes 30 seconds)
%cd /content/drive/MyDrive/workspace/nerfstudio/1/
!git clone https://github.com/NVIDIA/libglvnd
!sudo apt-get install libxext-dev libx11-dev x11proto-gl-dev
%cd libglvnd/
!sudo apt-get install autoconf automake libtool
!sudo apt-get install libffi-dev
!./autogen.sh
!./configure
!make  -j4
!sudo make install

# Installing COLMAP dependencies (takes 2 minutes)
%cd /content/drive/MyDrive/workspace/nerfstudio/1/
!sudo apt-get install \
    git \
    cmake \
    ninja-build \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libeigen3-dev \
    libflann-dev \
    libfreeimage-dev \
    libmetis-dev \
    libgoogle-glog-dev \
    libgtest-dev \
    libsqlite3-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libceres-dev

# Install nerfstudio
%cd /content/drive/MyDrive/workspace/nerfstudio/1/
!pip install git+https://github.com/nerfstudio-project/nerfstudio.git

/content/drive/MyDrive/workspace/nerfstudio/1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 76.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 138.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 114.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 159.5 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.5.0+cu121
    Uninstalling torch-2.5.0+cu121:
      Successfully uninstalled torch-2.5.0+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.20.0+cu121
    Uninstalling torchvision-0.20.0+cu121:
      Successfully uninsta

In [ ]:
# @markdown <h1> training code sample</h1>
#ns-viewer --viewer.make-share-url True --load-config /content/drive/MyDrive/workspace/nerfstudio/1/outputs/unnamed/splatfacto/2024-11-07_045704/config.yml
#ns-process-data video --data /content/drive/MyDrive/workspace/nerfstudio/1/data/nerfstudio/custom_data/ --output-dir /content/drive/MyDrive/workspace/nerfstudio/1/data/nerfstudio/custom_data/
#ns-process-data images --data /content/drive/MyDrive/workspace/nerfstudio/1/data/nerfstudio/custom_data/raw_images/ --output-dir /content/drive/MyDrive/workspace/nerfstudio/1/data/nerfstudio/custom_data/
#ns-train splatfacto --viewer.websocket-port 7007 --viewer.make-share-url True nerfstudio-data --data /content/drive/MyDrive/workspace/nerfstudio/1/data/nerfstudio/custom_data --downscale-factor 4

In [ ]:
# @markdown <h1> ターミナルはここでも起動できる</h1>
%cd /content/drive/MyDrive/workspace/nerfstudio/1/
!pip install colab-xterm
%load_ext colabxterm
%env TERM=xterm
from IPython.display import clear_output

clear_output(wait=True)
%xterm

In [ ]:
# @markdown <h1> Downloading and Processing Data</h1>
# @markdown <h3>Pick the preset scene or upload your own images/video</h3>
import glob
import os

from google.colab import files
from IPython.core.display import HTML, display

scene = "📤 upload your images"  # @param ['🖼 poster', '🚜 dozer', '🌄 desolation', '📤 upload your images' , '🎥 upload your own video', '🔺 upload Polycam data', '💽 upload your own Record3D data']
scene = " ".join(scene.split(" ")[1:])

if scene == "upload Polycam data":
    %cd /content/
    !mkdir -p /content/drive/MyDrive/workspace/nerfstudio/1/data/nerfstudio/custom_data/
    %cd /content/drive/MyDrive/workspace/nerfstudio/1/data/nerfstudio/custom_data/
    #%cd /content/data/nerfstudio/custom_data/
    uploaded = files.upload()
    dir = os.getcwd()
    if len(uploaded.keys()) > 1:
        print("ERROR, upload a single .zip file when processing Polycam data")
    dataset_dir = [os.path.join(dir, f) for f in uploaded.keys()][0]
    !ns-process-data polycam --data $dataset_dir --output-dir /content/drive/MyDrive/workspace/nerfstudio/1/data/nerfstudio/custom_data/
    #/content/data/nerfstudio/custom_data/
    scene = "custom_data"
elif scene == "upload your own Record3D data":
    display(HTML("<h3>Zip your Record3D folder, and upload.</h3>"))
    display(
        HTML(
            '<h3>More information on Record3D can be found <a href="https://docs.nerf.studio/en/latest/quickstart/custom_dataset.html#record3d-capture" target="_blank">here</a>.</h3>'
        )
    )
    %cd /content/
    !mkdir -p /content/data/nerfstudio/custom_data
    %cd /content/data/nerfstudio/custom_data/
    uploaded = files.upload()
    dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    record_3d_zipfile = preupload_datasets[0]
    !unzip $record_3d_zipfile -d /content/data/nerfstudio/custom_data
    custom_data_directory = glob.glob("/content/data/nerfstudio/custom_data/*")[0]
    !ns-process-data record3d --data $custom_data_directory --output-dir /content/data/nerfstudio/custom_data/
    scene = "custom_data"
elif scene in ["upload your images", "upload your own video"]:
    display(HTML("<h3>Select your custom data</h3>"))
    display(HTML("<p/>You can select multiple images by pressing ctrl, cmd or shift and click.<p>"))
    display(
        HTML(
            "<p/>Note: This may take time, especially on higher resolution inputs, so we recommend to download dataset after creation.<p>"
        )
    )
    !mkdir -p /content/data/nerfstudio/custom_data
    if scene == "upload your images":
        !mkdir -p /content/drive/MyDrive/workspace/nerfstudio/1/data/nerfstudio/custom_data/raw_images/
        #/content/data/nerfstudio/custom_data/raw_images
        %cd /content/drive/MyDrive/workspace/nerfstudio/1/data/nerfstudio/custom_data/raw_images/
        #/content/data/nerfstudio/custom_data/raw_images
        uploaded = files.upload()
        dir = os.getcwd()
    else:
        %cd /content/data/nerfstudio/custom_data/
        uploaded = files.upload()
        dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    del uploaded
    %cd /content/

    #f scene == "upload your images":
        #!unzip /content/drive/MyDrive/workspace/nerfstudio/1/data/nerfstudio/custom_data/raw_images/iloveimg-converted.zip -d /content/drive/MyDrive/workspace/nerfstudio/1/data/nerfstudio/custom_data/raw_images/
        #!ns-process-data images --data /content/drive/MyDrive/workspace/nerfstudio/1/data/nerfstudio/custom_data/raw_images/ --output-dir /content/drive/MyDrive/workspace/nerfstudio/1/data/nerfstudio/custom_data/
    #else:
        #video_path = preupload_datasets[0]
        #!ns-process-data video --data $video_path --output-dir /content/data/nerfstudio/custom_data/

    scene = "custom_data"
else:
    %cd /content/
    !ns-download-data nerfstudio --capture-name=$scene

print("Data Processing Succeeded!")

/content/drive/MyDrive/workspace/nerfstudio/1/data/nerfstudio/custom_data/raw_images


KeyboardInterrupt: 

In [ ]:
# @markdown <h1>※使わないnstall Nerfstudio and Dependencies (~8 min)</h1>

%cd /content/drive/MyDrive/workspace/nerfstudio/1/
!pip install --upgrade pip
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 --extra-index-url https://download.pytorch.org/whl/cu118

# Installing TinyCuda
%cd /content/drive/MyDrive/workspace/nerfstudio/1/
!gdown "https://drive.google.com/u/1/uc?id=1-7x7qQfB7bIw2zV4Lr6-yhvMpjXC84Q5&confirm=t"
!pip install tinycudann-1.7-cp310-cp310-linux_x86_64.whl

# Installing COLMAP
%cd /content/drive/MyDrive/workspace/nerfstudio/1/
!apt-get install colmap

# Install nerfstudio
%cd /content/drive/MyDrive/workspace/nerfstudio/1/
!pip install git+https://github.com/nerfstudio-project/nerfstudio.git

/content/drive/MyDrive/workspace/nerfstudio/1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 67.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 100.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 103.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 59.2 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.5.0+cu121
    Uninstalling torch-2.5.0+cu121:
      Successfully uninstalled torch-2.5.0+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.20.0+cu121
    Uninstalling torchvision-0.20.0+cu121:
      Successfully uninstal

In [ ]:
#scene
#!unzip /content/drive/MyDrive/workspace/nerfstudio/1/data/nerfstudio/custom_data/raw_images/iloveimg-converted.zip -d /content/drive/MyDrive/workspace/nerfstudio/1/data/nerfstudio/custom_data/raw_images/
#!ns-process-data images --data /content/drive/MyDrive/workspace/nerfstudio/1/data/nerfstudio/custom_data/raw_images/iloveimg-converted/ --output-dir /content/drive/MyDrive/workspace/nerfstudio/1/data/nerfstudio/custom_data/
#scene = "custom_data"
#print("Data Processing Succeeded!")

'custom_data'

In [ ]:
# @markdown <h1>※使わないStart Training</h1>

%cd /content/drive/MyDrive/workspace/nerfstudio/1/
!pip install colab-xterm
%load_ext colabxterm
%env TERM=xterm
from IPython.display import clear_output

clear_output(wait=True)
if os.path.exists(f"/content/drive/MyDrive/workspace/nerfstudio/1/data/nerfstudio/{scene}/transforms.json"):
    print(
        "\033[1m"
        + "Copy and paste the following command into the terminal window that pops up under this cell."
        + "\033[0m"
    )
    print(
        f"ns-train nerfacto --viewer.websocket-port 7007 --viewer.make-share-url True nerfstudio-data --data data/nerfstudio/{scene} --downscale-factor 4"
    )
    print()
    %xterm
else:
    from IPython.core.display import HTML, display

    display(HTML('<h3 style="color:red">Error: Data processing did not complete</h3>'))
    display(HTML("<h3>Please re-run `Downloading and Processing Data`, or view the FAQ for more info.</h3>"))